In [ ]:
https://developers.google.com/gmail/api/auth/web-server

credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'jupyter-api-ef5d0513e603.json', 
    'https://mail.google.com'
)
    
delegated_credentials = credentials.create_delegated('victor.tiendeo@gmail.com')

service = build(
    Config.get(SERVICE_ID, "build"),
    Config.get(SERVICE_ID, "version"),
    http = delegated_credentials.authorize(Http()),
    cache_discovery = False
)

In [1]:
import first_functions as gat

In [2]:
configs = gat.get_configs('config.ini')
gmail_service = gat.start_service(gat.get_configs('config.ini'), 'gmail')

HttpAccessTokenRefreshError: unauthorized_client: Client is unauthorized to retrieve access tokens using this method.

In [ ]:
gmail_service.users().labels().list(userId='victor.tiendeo@gmail.com').execute()

In [ ]:
dir(gmail_service)

In [ ]:
GET https://www.googleapis.com/gmail/v1/users/me/labels

In [ ]:
results = gmail_service.users().labels().list(userId='me').execute()

In [ ]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import base64
from apiclient import errors, discovery


def SendMessage(sender, to, subject, msgHtml, msgPlain, attachmentFile=None):
    service = gat.start_service(gat.get_configs('config.ini'), 'gmail')
    if attachmentFile:
        message1 = createMessageWithAttachment(sender, to, subject, msgHtml, msgPlain, attachmentFile)
    else: 
        message1 = CreateMessageHtml(sender, to, subject, msgHtml, msgPlain)
    result = SendMessageInternal(service, "me", message1)
    return result

def SendMessageInternal(service, user_id, message):
    try:
        message = (service.users().messages().send(userId=user_id, body=message).execute())
        print ('Message Id: %s' % message['id'])
        return message
    except errors.HttpError as error:
        print ('An error occurred: %s' % error)
        return "Error"
    return "OK"

def CreateMessageHtml(sender, to, subject, msgHtml, msgPlain):
    msg = MIMEMultipart('alternative')
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = to
    msg.attach(MIMEText(msgPlain, 'plain'))
    msg.attach(MIMEText(msgHtml, 'html'))
    return {'raw': base64.urlsafe_b64encode(msg.as_string())}

to = "victor.tiendeo@gmail.com"
sender = "victoor5@gmail.com"
subject = "subject"
msgHtml = "Hi<br/>Html Email"
msgPlain = "Hi\nPlain Email"

SendMessage(sender, to, subject, msgHtml, msgPlain)